## Data Driven Web Application Project 1
## Harshit Shrimali

## Kings County House price Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from pandas.plotting import scatter_matrix
from ipywidgets import Accordion, Button, Dropdown, FileUpload, HBox, Image, Output, Tab, VBox, Layout,Text
from PIL import Image as PilImage, ImageOps
from skimage import color
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import ipywidgets as widgets
from IPython.display import display

In [2]:
data= pd.read_csv('HousingPrices.csv')

In [3]:
#drop inappropriate Columns from dataFrame
data=data.drop(['id','date'], axis=1)

In [4]:
data['bathrooms'] = data['bathrooms'].round()
data['floors']= data['floors'].round()
data = data.astype({"bathrooms":'int', "floors":'int'})

In [5]:
df = data.groupby(['zipcode']).mean()

In [6]:
df = df[['lat','long']]
df.columns = ['latitude', 'longitude']

In [7]:
df = df.reset_index()

In [8]:
zipcode = data['zipcode'].unique()
zipcode.sort()

In [9]:
location = {}
for i in range(len(df)):
    location[df['zipcode'][i]] = [df['latitude'][i],df['longitude'][i]]

In [13]:
data['floors'].value_counts()

1    10680
2    10312
3      613
4        8
Name: floors, dtype: int64

In [ ]:
map = folium.Map(location=location[Zipcod_select], tiles='openstreetmap', zoom_start=8)
    for idx, row in dat[(dat.bedrooms == 2) & ((dat.bathrooms >= 1) & (dat.bathrooms < (1+1))) & (dat.condition == Condition_select) & ((dat.floors == Floors_select))].iterrows():
        Marker([row['lat'], row['long']]).add_to(map)
        popup_text = "Price: {}, Sqft: {}, Condition:{}, Beds: {}, Baths: {}, Floors: {}".format(row['price'],row['sqft_living'],row['condition'], row['bedrooms'],row['floors'],row['bathrooms'])
        popup = folium.Popup(popup_text, parse_html=True)
        marker = folium.Marker([row['lat'], row['long']], popup=popup).add_to(map)
display(map)

In [10]:
Map_button = widgets.Button(description='House Maps')
Map_button.style.button_color = 'lightblue'
House_graph = widgets.Button(description = 'Features Plot')
House_graph.style.button_color = 'lightyellow'
House_Price_Estimate = Button(description = 'Predict House Price')
House_Price_Estimate.style.button_color = 'lightgreen'
out=widgets.Output()
header1 = widgets.HBox([House_graph,Map_button,House_Price_Estimate])
display(widgets.VBox([header1,out]))

@out.capture()
def house():
    button = widgets.Button(description='Show Analysis',button_style = 'success')
    house_zip = widgets.Dropdown(description='ZipCode',options= [str(x) for x in zipcode])
    out1=widgets.Output()
    header2 = widgets.HBox([house_zip,button])
    display(widgets.VBox([header2,out1]))
    @out1.capture()
    def graph():
        dat = data[(data.zipcode == int(house_zip.value))][['price','yr_built']]
        da = (dat.groupby(['yr_built']).mean())
        da = da.reset_index()
        fig, ax = plt.subplots(2,2,figsize=(30,20));
        ax = plt.subplot(2,2,(1,2))
        axes = plt.gca()
        axes.set_xlim([1900,2020])
        ax.set_xticks(np.arange(1900, 2015,5.0))
        ax.set_title("Avg. Price as per build Year and No. of houses built per year", fontsize = 18)
        ax.set_xlabel('Year Built', fontsize=14)
        ax.set_ylabel('Average Price', fontsize=14)
        ax.plot(da['yr_built'],da['price'],'black',marker='o')
        for x,y in zip(da['yr_built'],da['price']):
            label = '{0:.2f} m'.format(y/1e6)
            plt.annotate(label,(x,y),textcoords="offset points",xytext=(0,10),ha='center',fontsize = 8)
        ax2 = ax.twinx()
        c = sns.histplot(dat['yr_built'],bins = len(dat['yr_built']))
        c.set_ylabel('Count of Houses', fontsize = 15)
        plt.subplot(2,2,3)
        plt.title("Bedroom Count Plot", fontsize = 18)
        a = sns.countplot(x= 'bedrooms', data=data[(data.zipcode == int(house_zip.value))])
        for container in a.containers:
            a.bar_label(container, fontsize = 12)
        a.set_xlabel('Bedrooms', fontsize = 15)
        a.set_ylabel('Count of Houses', fontsize = 15)
        a.set_xticklabels(a.get_xticks(), size = 15)
        plt.subplot(2,2,4)
        plt.title("Bathroom Count Plot", fontsize = 18)
        b= sns.countplot(x= 'bathrooms', data=data[(data.zipcode == int(house_zip.value))])
        for container in b.containers:
            b.bar_label(container, fontsize = 12)
        b.set_xlabel('Bathrooms', fontsize = 15)
        b.set_ylabel('Count of Houses', fontsize = 15)
        b.set_xticklabels(b.get_xticks(), size = 15)
        display(fig)
        plt.close(fig)
        
    def on_button_clicked(btn):
        out1.clear_output()
        graph()
            
    button.on_click(on_button_clicked)
    
@out.capture()    
def house_map(): 
    choice_zip = Dropdown(description='ZipCode',options= [str(x) for x in zipcode])
    No_Bed = Dropdown(description='No. Bedrooms',options= ['1','2','3','4','5'])
    No_Bath = Dropdown(description='No. Bathrooms',options= ['1','2','3'])
    No_Floors = Dropdown(description='No. Floors',options= ['1','2','3'])
    Condition = Dropdown(description='condition',options= ['1','2','3','4','5'])
    apply_button = Button(description='Map',button_style = 'success')
    out2= widgets.Output()
    header = HBox([choice_zip,No_Bed,No_Bath,No_Floors,Condition])
    display(widgets.VBox([header,apply_button,out2]))
    @out2.capture()
    def on_apply():
        map = folium.Map(location=location[int(choice_zip.value)], tiles='openstreetmap', zoom_start=11)
        for idx, row in data[(data.zipcode == int(choice_zip.value)) & (data.bedrooms == int(No_Bed.value)) & ((data.bathrooms >= int(No_Bath.value)) & (data.bathrooms < (int(No_Bath.value)+1))) & (data.condition == int(Condition.value)) & ((data.floors >= int(No_Floors.value)) & (data.floors < (int(No_Floors.value)+1)))].iterrows():
            Marker([row['lat'], row['long']]).add_to(map)
            popup_text = "Price: {}, Sqft: {}, Condition:{}, Beds: {}, Baths: {}, Floors: {}".format(row['price'],row['sqft_living'],row['condition'], row['bedrooms'],row['floors'],row['bathrooms'])
            popup = folium.Popup(popup_text, parse_html=True)
            marker = folium.Marker([row['lat'], row['long']], popup=popup).add_to(map)
        display(map)
    
    def Map_button(btn):
        out2.clear_output()
        on_apply()
        
    apply_button.on_click(Map_button)

@out.capture()    
def price_estimate():
    Price_prdiction = Button(description = 'Check Price',button_style = 'success')
    choice_zip1 = Dropdown(description='ZipCode',options= [str(x) for x in zipcode])
    No_Bed1 = Dropdown(description='No. Bedrooms',options= ['1','2','3','4','5'])
    No_Bath1 = Dropdown(description='No. Bathrooms',options= ['1','2','3','4'])
    No_Floors1 = Dropdown(description='No. Floors',options= ['1','2','3'])
    View = Dropdown(description='View',options= ['0','1','2','3','4'])
    Condition1 = Dropdown(description='Condition',options= ['1','2','3','4','5'])
    Grade = Dropdown(description='House Grade',options= ['4','5','6','7','8','9','10','11','12'])
    area_living = Text(description='Area',placeholder='Enter area in Square feet')
    out3 = widgets.Output()
    h1 = HBox([choice_zip1,No_Bed1,No_Bath1,No_Floors1])
    h2 = HBox([View,Condition1,Grade])
    h3 = HBox([area_living,Price_prdiction])
    display(widgets.VBox([h1,h2,h3,out3]))
    
    @out3.capture()
    def pred_price():
        df1 = data[['price', 'bedrooms', 'bathrooms', 'sqft_living', 'floors','view', 'condition', 'grade','zipcode']]
        npr= df1['price'].quantile(0.98)
        npr2 = df1['price'].quantile(0.02)
        df1 = df1[(df1['price'] <= npr) & (df1['price'] >= npr2)]
        x = df1.iloc[: , 1:].values
        y = df1.iloc[: , 0].values
        x_train,x_test,y_train,y_test=train_test_split(x, y, test_size=0.25, random_state=42)
        sc=StandardScaler()
        x_train=sc.fit_transform(x_train)
        x_test=sc.transform(x_test)
        RDregressor = RandomForestRegressor(n_estimators= 350, random_state=42)
        RDregressor.fit(x_train, y_train)
        Rd_pred = RDregressor.predict([[int(No_Bed1.value),int(No_Bath1.value),int(area_living.value),int(No_Floors1.value),int(View.value),int(Condition1.value),int(Grade.value),int(choice_zip1.value)]])
        Rd_pred = Rd_pred.item(0)
        print(f"House price will be around {Rd_pred:.2f}$")
        
    def predit_on_click(btn):
        out3.clear_output()
        pred_price()

    Price_prdiction.on_click(predit_on_click)
    
    

def house_on_click(btn):
    out.clear_output()
    house()
    
def Map_on_click(btn):
    out.clear_output()
    house_map()
    
def price_on_click(btn):
    out.clear_output()
    price_estimate()
    

House_graph.on_click(house_on_click)
Map_button.on_click(Map_on_click)
House_Price_Estimate.on_click(price_on_click)